In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("CKD.csv")

In [3]:
dataset

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.000000,76.459948,c,3,0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.000000,76.459948,c,2,0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.000000,76.459948,a,1,0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.000000,76.459948,d,1,0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.000000,50.000000,c,0,0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,a,0,0,normal,normal,notpresent,notpresent,219.000000,...,37.000000,9800.000000,4.400000,no,no,no,yes,poor,no,yes
395,51.492308,70.000000,c,0,2,normal,normal,notpresent,notpresent,220.000000,...,27.000000,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes
396,51.492308,70.000000,c,3,0,normal,normal,notpresent,notpresent,110.000000,...,26.000000,9200.000000,3.400000,yes,yes,no,poor,poor,no,yes
397,51.492308,90.000000,a,0,0,normal,normal,notpresent,notpresent,207.000000,...,38.868902,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset=dataset.astype(int) 

In [5]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [6]:
dataset['classification_yes'].value_counts()

classification_yes
1    249
0    150
Name: count, dtype: int64

In [7]:
independent=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]


In [8]:
dependent=dataset[['classification_yes']]

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [11]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV

param_grid={'solver':['newton-cg','lbfgs','liblinear','saga'],'penalty':['l2']}

grid=GridSearchCV(LogisticRegression(),param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [12]:
re=grid.cv_results_

In [13]:
y_pred=grid.predict(X_test)

In [14]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,y_pred)

In [15]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for best parameter{}".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter{'penalty': 'l2', 'solver': 'newton-cg'} 0.9924946382275899


In [16]:
print(cm)

[[51  0]
 [ 1 81]]


In [17]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        51
           1       1.00      0.99      0.99        82

    accuracy                           0.99       133
   macro avg       0.99      0.99      0.99       133
weighted avg       0.99      0.99      0.99       133



In [18]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

1.0

In [19]:
table=pd.DataFrame.from_dict(re)

In [20]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.019044,0.006266,0.002868,0.004474,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.981569,0.962264,0.981217,1.000000,1.000000,0.985010,0.014093,1
1,0.007019,0.008681,0.011061,0.007015,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.981569,0.962264,0.981217,1.000000,1.000000,0.985010,0.014093,1
2,0.007440,0.004185,0.001903,0.001675,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.963284,0.981233,0.962573,0.981217,0.981217,0.973905,0.008965,4
3,0.012834,0.002355,0.012125,0.002681,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.981569,0.981233,0.981217,0.981217,0.981217,0.981291,0.000139,3


In [21]:
age_input=int(input('Enter age:'))
Bp=float(input("Enter bp:"))
Al_input=int(input("Enter al 0 or 1:"))
Su_input=float(input("Enter su:"))
bgr_input=float(input("Enter bgr:"))
bu_input=float(input("Enter bu:"))
sc_input=float(input("enter sc:"))
sod_input=float(input("enter sod:"))
pot_input=float(input("enter pot:"))
hrmo_input=float(input("enter hrmo:"))
pcv_input=float(input("enter pcv:"))
wc_input=float(input("wc:"))
rc_input=float(input("rc:"))
sg_b_input=float(input("sg_b:"))
sg_c_input=float(input("sg_c:"))
sg_d_input=float(input("sg_d:"))
sg_e_input=float(input("sg_e:"))
rbc_normal_input=int(input("rbc_normal 0 or 1:"))
pc_normal_input=int(input("pc_normal 0 or 1:"))
pcc_present_input=int(input("pcc_present 0 or 1:"))
ba_present_input=int(input("ba_present 0 or 1:"))
htn_yes_input=int(input("htn_yes 0 or 1:"))
dm_yes_input=int(input("dm_yes 0 or 1:"))
cad_yes_input=int(input("cad_yes 0 or 1:"))
appet_yes_input=int(input("appet_yes 0 or 1:"))
pe_yes_input=int(input("pe_yes 0 or 1:"))
ane_yes_input=int(input("ane_yes 0 or 1:"))

Enter age:45
Enter bp:76.02
Enter al 0 or 1:1
Enter su:2.0
Enter bgr:143
Enter bu:25.30
enter sc:1.50000
enter sod:132.5324
enter pot:3.20000
enter hrmo:11.8000
enter pcv:36
wc:8400.513
rc:5.4
sg_b:32
sg_c:34
sg_d:36
sg_e:38
rbc_normal 0 or 1:0
pc_normal 0 or 1:1
pcc_present 0 or 1:0
ba_present 0 or 1:1
htn_yes 0 or 1:1
dm_yes 0 or 1:0
cad_yes 0 or 1:0
appet_yes 0 or 1:0
pe_yes 0 or 1:1
ane_yes 0 or 1:0


In [22]:
Future_Prediction=grid.predict([[age_input,Bp,Al_input,Su_input,bgr_input,bu_input,sc_input,sod_input,pot_input,hrmo_input,pcv_input,wc_input,rc_input,sg_b_input,sg_c_input,sg_d_input,sg_e_input,rbc_normal_input,pc_normal_input,pcc_present_input,ba_present_input,htn_yes_input,dm_yes_input,cad_yes_input,appet_yes_input,pe_yes_input,ane_yes_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
